# Homework 2 - Ally Hayden

# Part 1

In [1]:
import duckdb

conn = duckdb.connect("mimic_hw2.db")

# import all files
csv_files = {
    "admissions": "ADMISSIONS.csv",
    "icustays": "ICUSTAYS.csv",
    "patients": "PATIENTS.csv",
    "procedures_icd": "PROCEDURES_ICD.csv",
    "d_icd_procedures": "D_ICD_PROCEDURES.csv",
    "drgcodes": "DRGCODES.csv",
    "prescriptions": "PRESCRIPTIONS.csv"
}

for table_name, file_name in csv_files.items():
    query = f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM read_csv_auto('{file_name}');"
    conn.execute(query)


## Analysis Question 1

### 1A

In [2]:
conn.sql("""
SELECT
    a.ethnicity,
    p.drug_type,
    COUNT(*) AS total_prescriptions
FROM prescriptions p
JOIN admissions a ON p.hadm_id = a.hadm_id
GROUP BY a.ethnicity, p.drug_type
ORDER BY a.ethnicity, total_prescriptions DESC
""").df()

,ethnicity,drug_type,total_prescriptions
0,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,MAIN,200
1,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,BASE,80
2,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,ADDITIVE,2
3,ASIAN,MAIN,121
4,ASIAN,BASE,56
5,BLACK/AFRICAN AMERICAN,MAIN,476
6,BLACK/AFRICAN AMERICAN,BASE,169
7,HISPANIC OR LATINO,MAIN,226
8,HISPANIC OR LATINO,BASE,96
9,HISPANIC/LATINO - PUERTO RICAN,MAIN,860


### 1B-D

See .pdf file

## Analysis Question 2

### 2A

In [3]:
conn.sql("""
WITH base AS (
    SELECT
        p.subject_id,
        a.hadm_id,
        FLOOR(DATE_DIFF('day', p.dob, a.admittime) / 365.25) AS age,
        d.long_title AS procedure_name
    FROM admissions a
    JOIN patients p ON a.subject_id = p.subject_id
    JOIN procedures_icd pi ON a.hadm_id = pi.hadm_id
    JOIN d_icd_procedures d ON pi.icd9_code = d.icd9_code
),
age_grouped AS (
    SELECT *,
        CASE
            WHEN age <= 19 THEN '0-19'
            WHEN age BETWEEN 20 AND 49 THEN '20-49'
            WHEN age BETWEEN 50 AND 79 THEN '50-79'
            ELSE '>80'
        END AS age_group
    FROM base
),
ranked AS (
    SELECT
        age_group,
        procedure_name,
        COUNT(*) AS count,
        ROW_NUMBER() OVER (PARTITION BY age_group ORDER BY COUNT(*) DESC) AS rank
    FROM age_grouped
    GROUP BY age_group, procedure_name
)
SELECT age_group, procedure_name, count
FROM ranked
WHERE rank <= 3
ORDER BY age_group, rank;
""").df()


,age_group,procedure_name,count
0,0-19,"Venous catheterization, not elsewhere classified",3
1,0-19,Closure of skin and subcutaneous tissue of oth...,2
2,0-19,Other diagnostic procedures on brain and cereb...,1
3,20-49,"Venous catheterization, not elsewhere classified",8
4,20-49,Enteral infusion of concentrated nutritional s...,7
5,20-49,Continuous invasive mechanical ventilation for...,6
6,50-79,"Venous catheterization, not elsewhere classified",26
7,50-79,Enteral infusion of concentrated nutritional s...,22
8,50-79,Transfusion of packed cells,13
9,>80,"Venous catheterization, not elsewhere classified",19


### 2B-D

See .pdf file

## Analysis Question 3

### 3A

In [4]:
conn.sql("""
SELECT 
    p.gender,
    a.ethnicity,
    AVG(DATE_DIFF('day', i.intime, i.outtime)) AS avg_icu_stay_days,
    COUNT(*) AS num_stays
FROM icustays i
JOIN patients p ON i.subject_id = p.subject_id
JOIN admissions a ON i.hadm_id = a.hadm_id
WHERE i.intime IS NOT NULL AND i.outtime IS NOT NULL
GROUP BY p.gender, a.ethnicity
ORDER BY a.ethnicity,p.gender;
""").df()

,gender,ethnicity,avg_icu_stay_days,num_stays
0,M,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,11.500000,2
1,F,ASIAN,1.000000,1
2,M,ASIAN,7.000000,1
3,F,BLACK/AFRICAN AMERICAN,11.250000,4
4,M,BLACK/AFRICAN AMERICAN,3.000000,3
5,F,HISPANIC OR LATINO,7.333333,3
6,M,HISPANIC/LATINO - PUERTO RICAN,3.266667,15
7,F,OTHER,1.500000,2
8,M,OTHER,0.000000,1
9,M,UNABLE TO OBTAIN,14.000000,1


### 3B-D

See .pdf file

# Part 2

*No copies of the AWS crendentials file is stored on any publicly accessible location, nor is the file in any way shared with anyone outside of DATA_ENG 300 (Spring 2025).*

In [5]:
# %pip install cassandra-sigv4

In [6]:
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra import ConsistencyLevel
from ssl import SSLContext, CERT_REQUIRED
from cassandra_sigv4.auth import SigV4AuthProvider
import boto3

# SSL 
ssl_context = SSLContext()  # safer than using deprecated PROTOCOL_TLSv1_2
ssl_context.load_verify_locations('sf-class2-root.crt')
ssl_context.verify_mode = CERT_REQUIRED

# AWS 
boto_session = boto3.Session(region_name="us-east-2")
auth_provider = SigV4AuthProvider(boto_session)

# LOCAL_QUORUM 
execution_profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.LOCAL_QUORUM
)

cluster = Cluster(
    ['cassandra.us-east-2.amazonaws.com'],
    ssl_context=ssl_context,
    auth_provider=auth_provider,
    port=9142,
    execution_profiles={EXEC_PROFILE_DEFAULT: execution_profile}
)

session = cluster.connect()

/tmp/ipykernel_8554/2260347222.py:8: DeprecationWarning: ssl.SSLContext() without protocol argument is deprecated.
  ssl_context = SSLContext()  # safer than using deprecated PROTOCOL_TLSv1_2
/tmp/ipykernel_8554/2260347222.py:8: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  ssl_context = SSLContext()  # safer than using deprecated PROTOCOL_TLSv1_2


In [7]:
# create a set a keyspace
session.execute("""
CREATE KEYSPACE IF NOT EXISTS hw2_part2
WITH replication = {'class': 'SingleRegionStrategy'}
""")

In [8]:
session.set_keyspace('hw2_part2')

## Analysis Question 1

### 1A 

In [9]:
session.execute("""
CREATE TABLE IF NOT EXISTS drug_usage_by_ethnicity (
    ethnicity TEXT,
    drug_type TEXT,
    id UUID,
    dose_val_rx TEXT,
    PRIMARY KEY ((ethnicity), drug_type, id)
)
""")


### 1B

In [10]:
#join data using pandas
import pandas as pd

# relevant fields 
admissions = conn.sql("SELECT hadm_id, ethnicity FROM admissions").df()
prescriptions = conn.sql("SELECT hadm_id, drug_type, dose_val_rx FROM prescriptions").df()

merged = prescriptions.merge(admissions, on="hadm_id", how="left")


In [11]:
#insert sample rows into cassandra
import uuid

sample = merged.head(5000)

for _, row in sample.iterrows():
    session.execute("""
        INSERT INTO drug_usage_by_ethnicity (ethnicity, drug_type, id, dose_val_rx)
        VALUES (%s, %s, %s, %s)
    """, (
        str(row.ethnicity),
        str(row.drug_type),
        uuid.uuid4(),
        str(row.dose_val_rx)
    ))

### 1C

In [12]:
rows = session.execute("""
SELECT ethnicity, drug_type FROM drug_usage_by_ethnicity
""")

df = pd.DataFrame(rows.all())

In [13]:
summary = (
    df.groupby(["ethnicity", "drug_type"])
    .size()
    .reset_index(name="total_prescriptions")
    .sort_values(["ethnicity", "total_prescriptions"], ascending=[True, False])
)

top_usage = summary.groupby("ethnicity").head(1)

### 1D

In [14]:
summary.head()

,ethnicity,drug_type,total_prescriptions
1,ASIAN,MAIN,657
0,ASIAN,BASE,285
3,BLACK/AFRICAN AMERICAN,MAIN,1542
2,BLACK/AFRICAN AMERICAN,BASE,410
5,HISPANIC OR LATINO,MAIN,163


In [15]:
top_usage

,ethnicity,drug_type,total_prescriptions
1,ASIAN,MAIN,657
3,BLACK/AFRICAN AMERICAN,MAIN,1542
5,HISPANIC OR LATINO,MAIN,163
7,HISPANIC/LATINO - PUERTO RICAN,MAIN,860
9,OTHER,MAIN,98
12,UNABLE TO OBTAIN,MAIN,89
14,UNKNOWN/NOT SPECIFIED,MAIN,930
17,WHITE,MAIN,19300


## Analysis Question 2

### 2A

In [16]:
session.execute("""
CREATE TABLE IF NOT EXISTS procedures_by_age (
    age_group TEXT,
    procedure_name TEXT,
    id UUID,
    PRIMARY KEY ((age_group), procedure_name, id)
)
""")

### 2B

In [17]:
# relevant fields
admissions = conn.sql("SELECT subject_id, hadm_id, admittime FROM admissions").df()
patients = conn.sql("SELECT subject_id, dob FROM patients").df()
procedures = conn.sql("SELECT hadm_id, icd9_code FROM procedures_icd").df()
proc_titles = conn.sql("SELECT icd9_code, long_title FROM d_icd_procedures").df()

base = admissions.merge(patients, on="subject_id", how="left") \
                 .merge(procedures, on="hadm_id", how="inner") \
                 .merge(proc_titles, on="icd9_code", how="left")

# compute age
base["age"] = ((pd.to_datetime(base["admittime"]) - pd.to_datetime(base["dob"])) / pd.Timedelta(days=365.25)).astype(int)

# function to assign age
def assign_age_group(age):
    if age <= 19:
        return "0-19"
    elif age <= 49:
        return "20-49"
    elif age <= 79:
        return "50-79"
    else:
        return ">80"

base["age_group"] = base["age"].apply(assign_age_group)

sample = base[["age_group", "long_title"]].dropna().head(1000)

for _, row in sample.iterrows():
    session.execute("""
        INSERT INTO procedures_by_age (age_group, procedure_name, id)
        VALUES (%s, %s, %s)
    """, (
        row.age_group,
        row.long_title,
        uuid.uuid4()
    ))


### 2C

In [18]:
rows = session.execute("SELECT age_group, procedure_name FROM procedures_by_age")
df = pd.DataFrame(rows.all())

summary = (
    df.groupby(["age_group", "procedure_name"])
    .size()
    .reset_index(name="count")
    .sort_values(["age_group", "count"], ascending=[True, False])
)

top_3 = summary.groupby("age_group").head(3)


### 2D

In [19]:
top_3

,age_group,procedure_name,count
27,0-19,"Venous catheterization, not elsewhere classified",28
5,0-19,Closure of skin and subcutaneous tissue of oth...,8
0,0-19,"Application of external fixator device, femur",4
71,20-49,"Venous catheterization, not elsewhere classified",32
40,20-49,Enteral infusion of concentrated nutritional s...,28
37,20-49,Continuous invasive mechanical ventilation for...,24
168,50-79,"Venous catheterization, not elsewhere classified",104
101,50-79,Enteral infusion of concentrated nutritional s...,88
165,50-79,Transfusion of packed cells,52
233,>80,"Venous catheterization, not elsewhere classified",60


## Analysis Question 3

### 3A

In [20]:
session.execute("""
CREATE TABLE IF NOT EXISTS icu_stays_by_demo (
    gender TEXT,
    ethnicity TEXT,
    icu_stay_length DOUBLE,
    id UUID,
    PRIMARY KEY ((ethnicity), gender, id)
)
""")

### 3B

In [21]:
# relevant fields
icustays = conn.sql("SELECT subject_id, hadm_id, intime, outtime FROM icustays").df()
patients = conn.sql("SELECT subject_id, gender FROM patients").df()
admissions = conn.sql("SELECT hadm_id, ethnicity FROM admissions").df()

df = icustays.merge(patients, on="subject_id", how="left") \
             .merge(admissions, on="hadm_id", how="left")

df = df.dropna(subset=["intime", "outtime"])

# calculate ICU stay
df["icu_stay_length"] = (
    pd.to_datetime(df["outtime"]) - pd.to_datetime(df["intime"])
).dt.total_seconds() / (60 * 60 * 24)  # convert to days

sample = df[["gender", "ethnicity", "icu_stay_length"]].dropna().head(1000)

for _, row in sample.iterrows():
    session.execute("""
        INSERT INTO icu_stays_by_demo (ethnicity, gender, icu_stay_length, id)
        VALUES (%s, %s, %s, %s)
    """, (
        str(row.ethnicity),
        str(row.gender),
        float(row.icu_stay_length),
        uuid.uuid4()
    ))


### 3C

In [22]:
rows = session.execute("SELECT ethnicity, gender, icu_stay_length FROM icu_stays_by_demo")
icu_df = pd.DataFrame(rows.all())

summary = (
    icu_df.groupby(["gender", "ethnicity"])
    .agg(avg_icu_stay_days=("icu_stay_length", "mean"), num_stays=("icu_stay_length", "count"))
    .reset_index()
    .sort_values(["ethnicity", "gender"])
)

### 3D

In [23]:
summary

,gender,ethnicity,avg_icu_stay_days,num_stays
6,M,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,11.337159,6
0,F,ASIAN,0.662801,3
7,M,ASIAN,7.117303,3
1,F,BLACK/AFRICAN AMERICAN,11.201241,12
8,M,BLACK/AFRICAN AMERICAN,2.977245,9
2,F,HISPANIC OR LATINO,7.459637,9
9,M,HISPANIC/LATINO - PUERTO RICAN,3.243062,45
3,F,OTHER,1.336146,6
10,M,OTHER,0.105926,3
11,M,UNABLE TO OBTAIN,13.357037,3
